<a href="https://colab.research.google.com/github/yousenwang/langchain_llm/blob/main/retriever_results_to_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain==0.0.215 chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

https://github.com/nomic-ai/gpt4all/commit/213e033540fa3b68202bb12cf7f0134cfe6638aa?short_path=76b0339#diff-76b0339e6b9639ccaf2de9d1dde8b392e8e7bcd3cd514e625d5420239974af70

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.215
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
# from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [4]:
loader = DirectoryLoader('./', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [5]:
len(documents)

6

In [6]:
documents[0]

Document(page_content=' \n1 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure  \n \n \n \n \n \n \nwareconn standard operating procedure  \nRMA request  SOP  \n \nVersion : \nVersion  Date  Editor  Description  \nv 1.0  2023/03/16 Eric Sun First draft (English version)  \n \n \nContent  \nSOP ................................ ................................ ................................ ................................  2 \n1.1 RMA request procedure (Parts return)  ................................ ...............  2 \n \n \n \n \n', metadata={'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 0})

# Split Docs into Chunks

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000//2, chunk_overlap=200//2)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

12

In [9]:
texts[3]

Document(page_content='1.1 RMA request  procedure (Parts return)  \n1. System login  \n⚫ Module: https://www.wareconn.com/  \n⚫ Description:  Login  with account and password  \n \nTo log in, follow these steps:  \nGo to https://www.wareconn.com/ and click Log in  \n \nFill in your email and password, then click Login\n \nClick Enter  in the Customer portal', metadata={'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1})

# Embeddings

In [10]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

# Retriever

In [20]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [14]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [15]:
!pip install gpt4all==0.3.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.1 MB/s eta 0:00:00


In [16]:
llm_name = "ggml-gpt4all-j-v1.3-groovy.bin"

In [17]:

import gpt4all
gptj = gpt4all.GPT4All(llm_name)

100%|██████████| 3.79G/3.79G [01:30<00:00, 41.7MiB/s]


Model downloaded at:  /root/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin


In [27]:
def generate_from_template(context, query):
  return  f"""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  {context}

  Question: {query}
  Answer:"""


Q1 :How can I navigate to the RMA request page in the Wareconn Customer Portal? -3

A1: To navigate to the RMA request page, click on "Warranty Claims" in the left menu, then click on the "+Add" button.


In [21]:
q1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"
ans1 = retriever.get_relevant_documents(q1)

In [29]:
content_with_ans1 = ""
for i, a in enumerate(ans1):
  content_with_ans1+=a.page_content

prompt1 = generate_from_template(content_with_ans1, q1)
print(prompt1)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal3 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure  
2. Go to  RMA Request  Page 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims  
⚫ Description: Go to RMA request page  
 
To go to the RMA request page, follow these steps:  
Click Warranty Claims  in the left menu, then click +Add

  Question: How c

In [30]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res = gptj.generate(prompt1)
print(res)

 To navigate to the RMA request page in the Wareconn Customer Portal, click on "Warranty Claims" in the left menu and then click "+Add".
 To navigate to the RMA request page in the Wareconn Customer Portal, click on "Warranty Claims" in the left menu and then click "+Add".
CPU times: user 8min 23s, sys: 1.02 s, total: 8min 24s
Wall time: 5min 15s


In [31]:
res

' To navigate to the RMA request page in the Wareconn Customer Portal, click on "Warranty Claims" in the left menu and then click "+Add".'

Q2: What does the Reminder field indicate when submitting an RMA request? -6

Ans2: The Reminder field shows whether the RMA request is approvable (acceptable) for the warranty provider or not.

In [32]:
q2 = "What does the Reminder field indicate when submitting an RMA request?"
ans2 = retriever.get_relevant_documents(q2)

In [33]:
content_with_ans2 = ""
for i, a in enumerate(ans2):
  content_with_ans2+=a.page_content

prompt2 = generate_from_template(content_with_ans2, q2)
print(prompt2)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process6 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure  
5. Submit the RMA  request  
 
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit  
⚫ Description: Submit the RMA request  
 
The Reminder field will show you whether this RMA request is approvable

  Question: What does the Reminder field indicate when submitting an RMA request?
  Answer:


In [34]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res2 = gptj.generate(prompt2)

 The Reminder field indicates whether the RMA request is approvable for a warranty provider or not.
CPU times: user 7min 10s, sys: 1.08 s, total: 7min 11s
Wall time: 4min 26s


In [37]:
print(res2)

 The Reminder field indicates whether the RMA request is approvable for a warranty provider or not.


 Q3: What information should be filled in the Customer No. field when submitting an RMA request? -6

 Ans3: When submitting an RMA request, you should fill in the WAL# (Warranty Authorization Number) in the Customer No. field.

In [35]:
q3 = "What information should be filled in the Customer No. field when submitting an RMA request?"
ans3 = retriever.get_relevant_documents(q3)

In [36]:
content_with_ans3 = ""
for i, a in enumerate(ans3):
  content_with_ans3+=a.page_content

prompt3 = generate_from_template(content_with_ans3, q3)
print(prompt3)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal

  Question: What information should be filled in the Customer No. field when submitting an RMA request?
  Answer:


In [38]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res3 = gptj.generate(prompt3)

 The Customer No. field should be filled in with the customer's Wareconn account number.
CPU times: user 7min, sys: 1.13 s, total: 7min 1s
Wall time: 4min 21s


Q4: What steps should be followed when filling in the request information for an RMA? -5

Ans4: To fill in the request information for an RMA, follow these steps:
- Fill in the Product SN (Serial Number), Product PN (Part Number), and defective reason.
- Select the Rack PN (Part Number) by clicking on "Please select Parts" and choose the desired quantity.
- Click on "Confirm" to proceed.
- Optionally, fill in the Parts SN (Serial Number) for the failure part (not required).
- Click on "Save" to save the information.

In [39]:
q4 = "What steps should be followed when filling in the request information for an RMA?"
ans4 = retriever.get_relevant_documents(q4)

In [40]:
content_with_ans4 = ""
for i, a in enumerate(ans4):
  content_with_ans4+=a.page_content

prompt4 = generate_from_template(content_with_ans4, q4)
print(prompt4)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  The Reminder field will show you whether this RMA request is approvable  
(acceptable)  for warranty provider  or not.  
Please remember to fill in WAL# in Customer No. field  
Lastly, click Submit  in the top right corner to finish the  RMA requesting process1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal

  Question: What steps should be followed when filling in the request information for an RMA?
  Answer:


In [41]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res4 = gptj.generate(prompt4)

 When filling in the request information for an RMA, follow these steps:  
1. Enter the customer's Wareconn account number in the Customer No. field 
2. Enter a valid email address in the Email field 
3. Enter a valid password in the Password field 
4. Enter a valid phone number in the Phone field 
5. Enter a valid address in the Address field 
6. Enter a valid date in the Date field 
7. Enter a valid time in the Time field 
8. Enter a valid duration in the Duration field 
9. Enter a
CPU times: user 17min 14s, sys: 2.87 s, total: 17min 17s
Wall time: 10min 45s


Q5: Can this document be used for purposes other than introducing Wareconn functions and procedures? -3

Ans5: No, this document explicitly states that it should not be used for other purposes. Its sole intention is to introduce Wareconn functions and procedures.

In [42]:
q5 = "Can this document be used for purposes other than introducing Wareconn functions and procedures?"
ans5 = retriever.get_relevant_documents(q5)

In [43]:
content_with_ans5 = ""
for i, a in enumerate(ans5):
  content_with_ans5+=a.page_content

prompt5 = generate_from_template(content_with_ans5, q5)
print(prompt5)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  1 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure  
 
 
 
 
 
 
wareconn standard operating procedure  
RMA request  SOP  
 
Version : 
Version  Date  Editor  Description  
v 1.0  2023/03/16 Eric Sun First draft (English version)  
 
 
Content5 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  5 wareconn standard operating procedure  
4. Fill in request info  
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Fill in info  
⚫ Description:  
Fill in Product SN, Product PN

In [44]:
%%time
# You can run raw generate as well on your input. But performance will degrade.
res5 = gptj.generate(prompt5)

 No, this document can only be used to introduce Wareconn functions and procedures. It is not intended for any other purposes and should be used solely to introduce Wareconn functions and procedures.
CPU times: user 14min 36s, sys: 2.6 s, total: 14min 39s
Wall time: 9min 6s


https://docs.gpt4all.io/gpt4all_python.html

Fetch model list from https://gpt4all.io/models/models.json

In [45]:
gptj.list_models()

[{'order': 'a',
  'md5sum': '4acc146dd43eb02845c233c29289c7c5',
  'name': 'Hermes',
  'filename': 'nous-hermes-13b.ggmlv3.q4_0.bin',
  'filesize': '8136777088',
  'requires': '2.4.7',
  'ramrequired': '16',
  'parameters': '13 billion',
  'quant': 'q4_0',
  'type': 'LLaMA',
  'description': '<strong>Best overall model</strong><br><ul><li>Instruction based<li>Gives long responses<li>Curated with 300,000 uncensored instructions<li>Trained by Nous Research<li>Cannot be used commercially</ul>',
  'url': 'https://huggingface.co/TheBloke/Nous-Hermes-13B-GGML/resolve/main/nous-hermes-13b.ggmlv3.q4_0.bin',
  'promptTemplate': '### Instruction:\n%1\n### Response:\n'},
 {'order': 'b',
  'md5sum': '725f148218a65ce8ebcc724e52f31b49',
  'name': 'GPT4All Falcon',
  'filename': 'ggml-model-gpt4all-falcon-q4_0.bin',
  'filesize': '4061641216',
  'requires': '2.4.9',
  'ramrequired': '8',
  'parameters': '7 billion',
  'quant': 'q4_0',
  'type': 'Falcon',
  'description': '<strong>Best overall smaller 